In [ ]:
# import packages
import sys, os 
import pandas as pd
import numpy as np
np.seterr(divide='ignore',invalid='ignore')
import matplotlib.pyplot as plt
import json
import trodes2SS
import sungod_util

%load_ext autoreload
%autoreload 2



idx = pd.IndexSlice

In [ ]:
# cell to start up holoviews
import holoviews as hv

import scipy as sp

hv.extension('bokeh')

In [ ]:
# cell to test and make sure holoviews and bokeh are working
coords = np.random.rand(50,2)
points = hv.Points(coords)

points.opts(color='k', marker='+', size=10)

In [ ]:
# cell to read real-time hdf5 file

In [ ]:
# cell to fill in dataframes with realtime data

In [ ]:
#adjust timestamp in realtime decoder_data to match offline timestamps
decoder_data_adj_ts = decoder_data.copy()
decoder_data_adj_ts['adj_timestamp'] = decoder_data_adj_ts['timestamp']+1960000
#df1 = df1.assign(adj_timestamp=e.values)
decoder_data_adj_ts.shape

In [ ]:
#this is the most recent offline decode
file_path = '/data2/mcoulter/remy/remy_20_2_shuffle_0_posteriors_functionalized_150_decode_whole.h5'
hdf_base = '/analysis'
hdf_grps = 'decode/clusterless/offline/posterior'
hdf_label = 'sungod_trans_mat'

with pd.HDFStore(file_path, 'r') as store:
    main_path = os.path.join(hdf_base, hdf_grps, hdf_label)
    posteriors2 = store[main_path]

In [ ]:
# get linear position for offline

#### Define parameters
rat_name = 'remy'
day =  20     #previously:{'remy':[20], 'gus':[28], 'bernard':[23], 'fievel':[19]}
epoch = 2   # previously:{'remy':[4], 'gus':[2], 'bernard':[4], 'fievel':[2]} 

# define data source filepaths
path_base = '/data2/mcoulter/'
#path_base = '/mnt/vortex/mcoulter/'
raw_directory = path_base + 'raw_data/' + rat_name + '/'
linearization_path = path_base + 'maze_info/'
day_ep = str(day) + '_' + str(epoch)

#tetlist = None
tetlist = [4]
tetrodes= tetlist

pos_bin_size = 5
velocity_thresh_for_enc_dec = 4
velocity_buffer = 0

shift_amt_for_shuffle = 0

use_enc_as_dec = 1   #if you want to use the encoding marks as the decoding marks

discrete_tm_val=.99   # for classifier

# IMPORT and process data

#initialize data importer
datasrc = trodes2SS.TrodesImport(raw_directory, rat_name, [day], 
                       [epoch], tetrodes)

# Import trials
trials = datasrc.import_trials()

# Import raw position
linear_pos_raw = datasrc.import_pos(xy='x')   # this is basically just to pull in speed, will be replaced with linearized below
#posY = datasrc.import_pos(xy='y')          #  OPTIONAL; useful for 2d visualization

# if linearization exists, load it. if not, run the linearization.
lin_output1 = linearization_path + rat_name + '/' + rat_name + '_' + day_ep + '_' + 'linearized_distance.npy'
lin_output2 = linearization_path + rat_name + '/' + rat_name + '_' + day_ep + '_' + 'linearized_track_segments.npy'

if os.path.exists(lin_output1) == False:
    print('Linearization result doesnt exist. Doing linearization calculation')
    sungod_util.run_linearization_routine(rat_name, day, epoch, linearization_path, raw_directory, gap_size=20)
    linear_pos_raw['linpos_flat'] = np.load(lin_output1)
    track_segment_ids = np.load(lin_output2)
else: 
    print('Linearization found. Loading it')
    linear_pos_raw['linpos_flat'] = np.load(lin_output1)   #replace x pos with linerized 
    track_segment_ids = np.load(lin_output2)

# generate boundary definitions of each segment
arm_coords, _ = sungod_util.define_segment_coordinates(linear_pos_raw, track_segment_ids)  # optional addition output of all occupied positions (not just bounds)

#bin linear position 
binned_linear_pos, binned_arm_coords, pos_bins = sungod_util.bin_position_data(linear_pos_raw, arm_coords, pos_bin_size)


In [ ]:
# add linear position to offline posterior table
post_to_merge1 = posteriors2.copy()
post_to_merge1.reset_index(inplace=True)
post_to_merge1['timestamp1']=post_to_merge1['timestamp']
pos_vel = binned_linear_pos.copy()
pos_vel1 = pos_vel.reset_index()
pos_vel1['timestamp1'] = pos_vel1['timestamp']
pos_vel2 = pos_vel1.drop(['day','epoch','time','timestamp'], axis=1)
#pos_vel2
posterior_with_pos_vel = pd.merge_asof(post_to_merge1,pos_vel2,on='timestamp1',direction='nearest')


In [ ]:
#create posterior_to_plot
#posterior_offline = posteriors1.reset_index()
posterior_offline = posteriors2.reset_index()

# end of session - compared to real-time looking for arm 2 bias 154653600-157081200
#posterior_to_plot = posterior_offline[(posterior_offline['timestamp'] > 154653600+1960000) & (posterior_offline['timestamp'] < 157081200+1960000)]

#this is a nice run down arm 7
#posterior_to_plot = posterior_offline[(posterior_offline['timestamp'] > 47598850) & (posterior_offline['timestamp'] < 48921150)]

#this is a nice run down arm 5
#posterior_to_plot = posterior_offline[(posterior_offline['timestamp'] > 103960800) & (posterior_offline['timestamp'] < 104769750)]
#offline_pos_to_plot = posterior_with_pos_vel[(posterior_with_pos_vel['timestamp1'] > 103960800) & (posterior_with_pos_vel['timestamp1'] < 104769750)]

#posterior_to_plot = posterior_offline[(posterior_offline['time'] > 3742.45) & (posterior_offline['time'] < 3742.55)]
# whole trial 60
#posterior_to_plot = posterior_offline[(posterior_offline['time'] > 3715) & (posterior_offline['time'] < 3755)]

#trial 60 replay 1
posterior_to_plot = posterior_offline[(posterior_offline['time'] > 3729) & (posterior_offline['time'] < 3730.4)]

print(posterior_to_plot.shape)

In [ ]:
#create decoder_to_plot
decoder_data_adj_ts = decoder_data.copy()
decoder_data_adj_ts['adj_timestamp'] = decoder_data_adj_ts['timestamp']+1960000

stim_lockout_adj_ts = stim_lockout.copy()
stim_lockout_adj_ts['adj_timestamp'] = stim_lockout_adj_ts['timestamp']+1960000

# shadow trials of arm 6 116141100 - 120145200
#decoder_to_plot = decoder_data_adj_ts[(decoder_data_adj_ts['timestamp'] > 116141100) & (decoder_data_adj_ts['timestamp'] < 120145200)]

# end of session strong bias to arm 2 154653600-157081200
#decoder_to_plot = decoder_data_adj_ts[(decoder_data_adj_ts['timestamp'] > 154653600) & (decoder_data_adj_ts['timestamp'] < 157081200)]

# a nice run down arm 7 - hmmm this doesnt look clear on the realtime decoder
#posterior_to_plot = posterior_offline[(posterior_offline['timestamp'] > 47598850) & (posterior_offline['timestamp'] < 48921150)]
#decoder_to_plot = decoder_data_adj_ts[(decoder_data_adj_ts['adj_timestamp'] > 47598850) & (decoder_data_adj_ts['adj_timestamp'] < 48921150)]

print(decoder_to_plot.shape)
print(stim_to_plot.shape)

In [ ]:
# get ripple times from offline posteriors dataframe
ripple_times = posterior_to_plot.index[posterior_to_plot['ripple_grp'] > 0].tolist()
ripple_timestamps = posterior_to_plot[posterior_to_plot['ripple_grp'] > 0]['timestamp'].values - 1957500
#closest value lookup, return row
decoder_ripple_times = decoder_to_plot.iloc[(decoder_to_plot['timestamp']-ripple_timestamps[0]).abs().argsort()[:1]]
for i in np.arange(1,ripple_timestamps.shape[0]):
    ripple_row = decoder_to_plot.iloc[(decoder_to_plot['timestamp']-ripple_timestamps[i]).abs().argsort()[:1]]
    decoder_ripple_times = decoder_ripple_times.append(ripple_row)
    #print(i)
decoder_ripple_times.shape
#decoder_ripple_times.index.tolist()

In [ ]:
# offline posteriors, 150 uV threshold
# trial 60 - middle ripples - larger range is whole trial
#offline arm ends 8, 25, 42, 59, 77, 94, 111, 128, 145 
import seaborn as sns
sns.set()

#posterior_offline = posteriors1.reset_index()

#ripple_times = posterior_to_plot.index[posterior_to_plot['ripple_grp'] > 0].tolist()

#ripple_times = posterior_to_plot.index[posterior_to_plot['ripple_grp'] == 1386].tolist()
#ripple_start = ripple_times[0] - posterior_to_plot.index[0]
#print(ripple_start)
#ripple_end = ripple_times[-1] - posterior_to_plot.index[0]
#print(ripple_end)

posterior_offline1 = posterior_to_plot.drop(['day','epoch','timestamp','time','num_spikes','dec_bin','ripple_grp'], axis=1)
#posterior_offline1 = posterior_offline.drop(['day','epoch','timestamp','time','num_spikes','dec_bin','ripple_grp'], axis=1)
posterior_offline2 = posterior_offline1.fillna(0)
post_heatmap = posterior_offline2.transpose()

# to only plot a few position bins
#post_heatmap = posterior_offline2.transpose()[0:20]

#post_heatmap = posterior_offline2[425005:426825].transpose()
#replay 1
#post_heatmap = posterior_only[486533:486733].transpose()
#replay 2
#post_heatmap = posterior_only[487356:487776].transpose()

post_heatmap = post_heatmap.iloc[::-1]

plt.figure(figsize=(15,5))
ax = (sns.heatmap(post_heatmap,vmin=0, vmax=0.7))
ax.hlines([145-10,145-27,145-44,145-61,145-79,145-96,145-113,145-130], lw=2, color='w',*ax.get_xlim())
#ax.scatter(np.arange(0,offline_pos_to_plot.shape[0]),145-offline_pos_to_plot['linpos_flat'].values,s=1,alpha=0.05,color='cyan')
#ax.vlines([ripple_times-posterior_to_plot.index[0]],lw=1,color='w',alpha=0.2,*ax.get_ylim())

#ax.vlines([ripple_start],lw=2,color='w',alpha=0.3,*ax.get_ylim())
#ax.vlines([ripple_end],lw=2,color='w',alpha=0.3,*ax.get_ylim())
#sns.heatmap(post_hist1[50000:55000],vmin=0, vmax=0.5)

In [ ]:
# all tetrodes (21) for full session on drought, 150 uV threshold, good linearization, adjusted TS
# 20 tetrodes for full session on drought, 150 uV threshold, adjusted timestamps
# realtime arm ends: 1, 17, 33, 49, 65, 81, 97, 113, 129
# new arm ends: 7, 23, 39, 55, 71, 87, 103, 119, 135
import seaborn as sns
sns.set()

#ripple_times = posterior_to_plot.index[posterior_to_plot['ripple_grp'] > 0].tolist()
#this value is currently just emprically set
#ripple_timestamps = posterior_to_plot[posterior_to_plot['ripple_grp'] > 0]['timestamp'].values - 1958800
#closest value lookup, return row
#decoder_ripple_times = decoder_to_plot.iloc[(decoder_to_plot['timestamp']-ripple_timestamps[0]).abs().argsort()[:1]]
#for i in np.arange(1,ripple_timestamps.shape[0]):
#    ripple_row = decoder_to_plot.iloc[(decoder_to_plot['timestamp']-ripple_timestamps[i]).abs().argsort()[:1]]
#    decoder_ripple_times = decoder_ripple_times.append(ripple_row)
    #print(i)
#decoder_ripple_times
#decoder_ripple_list = decoder_ripple_times.index.tolist()

posterior_only = decoder_to_plot.drop(['rec_ind','timestamp','real_pos_time','real_pos','spike_count','adj_timestamp'], axis=1)

post_heatmap = posterior_only.transpose()
post_heatmap = post_heatmap.iloc[::-1]

plt.figure(figsize=(15,5))
ax = (sns.heatmap(post_heatmap,vmin=0, vmax=0.7))
#gap lines need to be inverse of where you would expect
ax.hlines([135-9,135-25,135-41,135-57,135-73,135-89,135-105,135-121], lw=2, color='w',*ax.get_xlim())
ax.scatter(np.arange(0,decoder_to_plot.shape[0]),135-decoder_to_plot['real_pos'].values,s=1,alpha=0.5,color='cyan')
#ax.scatter(stim_lockout['timestamp'].values,(stim_lockout['lockout_state'].values)*50,s=2,alpha=1)
#ax.vlines([decoder_ripple_list-decoder_to_plot.index[0]],lw=1,color='w',alpha=0.2,*ax.get_ylim())

In [ ]:
#plot realtime data no ripple
import seaborn as sns
sns.set()

posterior_only = decoder_to_plot.drop(['rec_ind','timestamp','real_pos_time','real_pos','spike_count','adj_timestamp'], axis=1)

post_heatmap = posterior_only.transpose()
post_heatmap = post_heatmap.iloc[::-1]

plt.figure(figsize=(15,5))
ax = (sns.heatmap(post_heatmap,vmin=0, vmax=0.7))
#gap lines need to be inverse of where you would expect
ax.hlines([135-9,135-25,135-41,135-57,135-73,135-89,135-105,135-121], lw=2, color='w',*ax.get_xlim())